# This notebook is an extension of loan_ml-train_with_investor_features.ipynb just with keras

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('lending_club_ml.csv')
df.drop(['id'], inplace=True, axis=1)

C:\Users\leero\Anaconda3\envs\tf_gpu\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# investor only features
df_lc = df[['loan_status', 'loan_amnt', 'purpose', 'sub_grade', 'int_rate', 'term',  'installment',
            'home_ownership', 'emp_length',  'annual_inc', 'dti',
            'fico_range_low', 'fico_range_high', 'open_acc', 'total_acc', 'revol_bal', 'revol_util', 'inq_last_6mths', 'acc_now_delinq',
            'delinq_amnt', 'delinq_2yrs', 'mths_since_last_delinq', 'pub_rec', 'mths_since_last_record', 'mths_since_last_major_derog', 'collections_12_mths_ex_med']]

In [4]:
# dummy variables
# I have decided to use subgrade and not grade as these are suppose to be similar features but, subgrade is more granular

df_lc = pd.concat([df_lc, pd.get_dummies(
    df_lc.sub_grade, prefix='sub_grade', drop_first=True)], axis=1)
df_lc = pd.concat([df_lc, pd.get_dummies(df_lc.home_ownership,
                                         prefix='home_ownership', drop_first=True)], axis=1)
df_lc = pd.concat([df_lc, pd.get_dummies(
    df_lc.purpose, prefix='purpose', drop_first=True)], axis=1)

df_lc.drop(columns=['sub_grade', 'home_ownership', 'purpose'], inplace=True)

In [5]:
# fill all nans with 0
df_lc.fillna(0, inplace=True)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df_lc.drop(['loan_status'], axis=1)
y = df_lc.loan_status


scaler = StandardScaler()

X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

def score(x,y,model):
    print(np.mean(y))
    print(confusion_matrix(y, model.predict_classes(x)))
    print(classification_report(y, model.predict_classes(x)))

#def precision(y_true, y_pred):
#    return tf.keras.backend.precision(y_true, y_pred)


In [8]:
import tensorflow as tf
from tensorflow import keras

x_len = len(X.columns)

some_activation = tf.keras.layers.LeakyReLU(alpha=0.3)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(x_len, activation=some_activation, input_shape=(x_len,)))
model.add(tf.keras.layers.Dense(1000, activation=some_activation))
model.add(tf.keras.layers.Dense(2000, activation=some_activation))
model.add(tf.keras.layers.Dense(2000, activation=some_activation))
model.add(tf.keras.layers.Dense(2500, activation=some_activation))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy','categorical_accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.fit(Xn_train, yn_train, validation_split=0.25, epochs=5)
print(np.mean(yn_train))

Train on 1316037 samples, validate on 438679 samples
Epoch 1/5
1316037/1316037 [==============================] - 268s 204us/sample - loss: 0.4545 - acc: 0.8026 - categorical_accuracy: 0.0133 - val_loss: 0.4525 - val_acc: 0.8040 - val_categorical_accuracy: 0.0346
Epoch 2/5
1316037/1316037 [==============================] - 265s 202us/sample - loss: 0.4514 - acc: 0.8036 - categorical_accuracy: 0.0222 - val_loss: 0.4503 - val_acc: 0.8047 - val_categorical_accuracy: 0.0253
Epoch 3/5
1316037/1316037 [==============================] - 268s 203us/sample - loss: 0.4504 - acc: 0.8042 - categorical_accuracy: 0.0251 - val_loss: 0.4500 - val_acc: 0.8050 - val_categorical_accuracy: 0.0288
Epoch 4/5
1316037/1316037 [==============================] - 267s 203us/sample - loss: 0.4496 - acc: 0.8045 - categorical_accuracy: 0.0262 - val_loss: 0.4491 - val_acc: 0.8049 - val_categorical_accuracy: 0.0248
Epoch 5/5
1316037/1316037 [==============================] - 266s 202us/sample - loss: 0.4488 - acc: 0.

In [10]:
score(Xn_test,yn_test,model)

0.8009672589365736
[[  8971 140706]
 [  6820 595525]]
              precision    recall  f1-score   support

           0       0.57      0.06      0.11    149677
           1       0.81      0.99      0.89    602345

    accuracy                           0.80    752022
   macro avg       0.69      0.52      0.50    752022
weighted avg       0.76      0.80      0.73    752022



In [11]:
model.evaluate(Xn_test, yn_test)

752022/752022 [==============================] - 49s 65us/sample - loss: 0.4507 - acc: 0.8038 - categorical_accuracy: 0.0210


[0.4506883563272606, 0.8038275, 0.020998055]